In [6]:


from PIL import Image
import h5py
import anubisPlotUtils as anPlot
import json
import numpy as np
import os
import hist as hi
import matplotlib
import matplotlib.pyplot as plt
from itertools import combinations
import matplotlib.colors as colors
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc.
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import sys
import AnalysisToolAnubis as AT
import seaborn as sns
import pandas as pd
import tkinter as tk
from tkinter import ttk
from pandasgui import show
from matplotlib.patches import Circle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import convolve2d
from scipy.ndimage import label, find_objects
from scipy.spatial.distance import pdist, squareform
from matplotlib.backends.backend_pdf import PdfPages
import ipywidgets as widgets
from IPython.display import display
import sys
from scipy.stats import gaussian_kde
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.ticker import MaxNLocator
import AnubisSuperScript as ass
from functools import partial
import importlib
importlib.reload(ass)

ERROR:root:Cannot activate multiple GUI eventloops


<module 'AnubisSuperScript' from 'c:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\Scripts\\AnubisSuperScript.py'>

In [7]:

mapping = {
    0: {
        (0, 31): ('rpc0', 'eta'),
        (32, 63): ('rpc0', 'phi1'),
        (64, 95): ('rpc0', 'phi2'),
        (96, 127): ('rpc1', 'eta'),
    },
    1: {
        (0, 31): ('rpc1', 'phi1'),
        (32, 63): ('rpc1', 'phi2'),
        (64, 95): ('rpc2', 'eta'),
        (96, 127): ('rpc2', 'phi1'),
    },
    2: {
        (0, 31): ('rpc2', 'phi2'),
        (32, 63): ('rpc3', 'eta'),
        (64, 95): ('rpc3', 'phi1'),
        (96, 127): ('rpc3', 'phi2'),
    },
    3: {
        (0, 31): ('rpc4', 'eta'),
        (32, 63): ('rpc4', 'phi1'),
        (64, 95): ('rpc4', 'phi2'),
        (96, 127): ('rpc5', 'eta'),
    },
    4: {
        (0, 31): ('rpc5', 'phi1'),
        (32, 63): ('rpc5', 'phi2'),
    },
}

strips_on_steroid = {
    'rpc0': {'phi': [0]},
    'rpc1': {'phi': [0]},
    'rpc2': {'phi': [0]},
    'rpc3': {'phi': [0]},
    'rpc4': {'eta': [31,0], 'phi': [0]},
    'rpc5': {'eta': [31,0], 'phi': [0]},
}


In [8]:
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\60sRun_24_3_4.h5')
df = ass.remake_data(thisData, mapping)

In [9]:
cluster_df = ass.calculate_cluster_metrics_better(df, 5)

In [10]:
remove_noisy_strips_with_map = partial(ass.remove_noisy_strips, noisy_strips=strips_on_steroid)
anti_steriod_df = cluster_df[cluster_df.apply(lambda row: remove_noisy_strips_with_map(row), axis=1)]

In [11]:
fully_massaged_df = anti_steriod_df[anti_steriod_df['size'] <= 5]

In [ ]:
show(anti_steriod_df)

In [25]:
show(fully_massaged_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [21]:
show(cluster_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [7]:
pdf_path1 = 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\OG3\\rpc_cluster_analysis_all.pdf'
pdf_path2 = 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\OG3\\rpc_cluster_analysis_anti_steriod.pdf'
# ass.plot_rpc_histograms(cluster_df, pdf_path1)
ass.plot_rpc_histograms(anti_steriod_df, pdf_path2)

In [8]:
pdf_path3 = 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\OG3\\rpc_cluster_analysis_by_size.pdf'
ass.plot_rpc_histograms_by_cluster_size(anti_steriod_df, pdf_path3)

In [10]:
pdf_path4 = 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\OG3\\rpc_cluster_time_analysis.pdf'
fully_massaged_df_with_time = fully_massaged_df[fully_massaged_df['end_time'] <= 350]
ass.plot_time_differences_and_event_times(fully_massaged_df_with_time, pdf_path4)

KeyboardInterrupt: 

In [12]:
fully_massaged_df_with_time = fully_massaged_df[fully_massaged_df['end_time'] <= 350]

In [9]:
show(fully_massaged_df_with_time)

NameError: name 'fully_massaged_df_with_time' is not defined

In [11]:
def calculate_metric_for_combo(combo, rpc_heights):
    locations = np.array([c['location'] for c in combo])
    heights = np.array([rpc_heights[c['rpc']] for c in combo])
    
    if len(locations) > 1 and np.std(locations) > 0 and np.std(heights) > 0:
        try:
            coeffs, cov = np.polyfit(locations, heights, 1, cov=True)
        except np.linalg.LinAlgError:
            coeffs = [np.inf, np.inf]
            cov = np.array([[np.inf, np.inf], [np.inf, np.inf]])
            
        slope, intercept = coeffs
        slope_error, intercept_error = np.sqrt(np.diag(cov))
        predicted = slope * locations + intercept
        residuals = heights - predicted
        RSS = np.sum(residuals ** 2)
    else:
        slope = intercept = slope_error = intercept_error = RSS = np.inf
    
    return slope, intercept, slope_error, intercept_error, RSS, combo

def analyze_inter_rpc_hit_with_timing_adjusted(df):
    rpc_time_offsets = {
    ('rpc0', 'eta'): (7.94, 12.48),
    ('rpc0', 'phi'): (-2.38, 13.69),
    ('rpc1', 'eta'): (8.36, 12.22),
    ('rpc1', 'phi'): (-3.79, 13.25),
    ('rpc2', 'eta'): (8.84, 12.56),
    ('rpc2', 'phi'): (-4.35, 13.57),
    ('rpc3', 'eta'): (6.86, 12.41),
    ('rpc3', 'phi'): (-4.3, 13.96),
    ('rpc4', 'eta'): (2.7, 12.37),
    ('rpc4', 'phi'): (-7.89, 13.41),
    ('rpc5', 'eta'): (2.82, 13.05),
    ('rpc5', 'phi'): (9.15, 14.14),
}
    paths = []

    adjusted_muon_speed_cm_ns = 28

    rpc_heights = {
        'rpc0': 0, 
        'rpc1': 0.5, 
        'rpc2': 1.0, 
        'rpc3': 61.5, 
        'rpc4': 121.5, 
        'rpc5': 122.0
    }

    for event_number, event_group in df.groupby('event_number'):
        for direction in ['eta', 'phi']:
            direction_group = event_group[event_group['strip_direction'] == direction]
            all_clusters = []

            unique_rpcs = direction_group['rpc_number'].unique()
            for rpc in unique_rpcs:
                rpc_group = direction_group[direction_group['rpc_number'] == rpc]
                for _, row in rpc_group.iterrows():
                    location_scaling = 3.09375 if direction == 'eta' else 2.8125
                    strip_locations = np.array(row['locations'])
                    non_zero_locations = strip_locations[strip_locations != 0]
                    if non_zero_locations.size > 0:
                        strip_location = non_zero_locations[0] 
                    else:
                        continue 

                    location = strip_location * location_scaling
                    event_time = np.mean(row['times']) - rpc_time_offsets[(rpc, direction)][0]
                    cluster_size_scaled = max(row['size'] * location_scaling, location_scaling) / 2
                    all_clusters.append({
                        'rpc': rpc,
                        'location': location,
                        'event_time': event_time,
                        'uncertainty': cluster_size_scaled,
                        'original_location': strip_location
                    })

            combination_metrics = []
            valid_combinations = [] 
            for n in range(3, 6):
                for combo in combinations(all_clusters, n):
                    metric = calculate_metric_for_combo(combo, rpc_heights)
                    if metric[4] != np.inf:
                        combination_metrics.append(metric)
                        
            # Move filtering logic outside the loop so it's not reset each time
            for combo_metric in combination_metrics:
                combo = combo_metric[-1]
                if len({c['rpc'] for c in combo}) < len(combo):
                    continue

                time_diffs_are_valid = True
                for i in range(len(combo)):
                    for j in range(i + 1, len(combo)):
                        error_window = rpc_time_offsets[(combo[i]['rpc'], direction)][1] + rpc_time_offsets[(combo[j]['rpc'], direction)][1]
                        
                        # Use direct height differences
                        height_diff = abs(rpc_heights[combo[i]['rpc']] - rpc_heights[combo[j]['rpc']])
                        
                        time_diff = abs(combo[i]['event_time'] - combo[j]['event_time'])
                        expected_time_diff = height_diff / adjusted_muon_speed_cm_ns

                        uncertainty_margin = 5
                        # Use the expected time difference with the error window and uncertainty margin for validation
                        if not (time_diff <= expected_time_diff + error_window + uncertainty_margin):
                            time_diffs_are_valid = False
                            break
                    if not time_diffs_are_valid:
                        break

                if time_diffs_are_valid:
                    valid_combinations.append(combo_metric)

            for valid_combination in valid_combinations:
                paths.append({
                    'Event Number': event_number,
                    'Direction': direction,
                    'Slope': valid_combination[0],
                    'Intercept': valid_combination[1],
                    'Slope_error': valid_combination[2],
                    'Intercept_error': valid_combination[3],
                    'Used Coordinates': [(c['rpc'], c['original_location'], c['event_time']) for c in valid_combination[-1]],
                    'RSS': valid_combination[4],
                })

    path_df = pd.DataFrame(paths)
    return path_df

In [20]:
from tqdm import tqdm
def calculate_metric_for_combo(combo, rpc_heights):
    locations = np.array([c['location'] for c in combo])
    heights = np.array([rpc_heights[c['rpc']] for c in combo])
    
    if len(heights) > 1 and np.std(locations) > 0 and np.std(heights) > 0:
        try:
            coeffs, cov = np.polyfit(heights, locations, 1, cov=True)  
        except np.linalg.LinAlgError:
            coeffs = [np.inf, np.inf]
            cov = np.array([[np.inf, np.inf], [np.inf, np.inf]])
            
        slope, intercept = coeffs
        slope_error, intercept_error = np.sqrt(np.diag(cov))
        predicted = slope * heights + intercept 
        residuals = locations - predicted
        RSS = np.sum(residuals ** 2)
    else:
        slope = intercept = slope_error = intercept_error = RSS = np.inf
    
    return slope, intercept, slope_error, intercept_error, RSS, combo

def analyze_inter_rpc_hit_with_timing_adjusted(df):
    rpc_time_offsets = {
    ('rpc0', 'eta'): (7.94, 12.48),
    ('rpc0', 'phi'): (-2.38, 13.69),
    ('rpc1', 'eta'): (8.36, 12.22),
    ('rpc1', 'phi'): (-3.79, 13.25),
    ('rpc2', 'eta'): (8.84, 12.56),
    ('rpc2', 'phi'): (-4.35, 13.57),
    ('rpc3', 'eta'): (6.86, 12.41),
    ('rpc3', 'phi'): (-4.3, 13.96),
    ('rpc4', 'eta'): (2.7, 12.37),
    ('rpc4', 'phi'): (-7.89, 13.41),
    ('rpc5', 'eta'): (2.82, 13.05),
    ('rpc5', 'phi'): (9.15, 14.14),
}
    paths = []

    adjusted_muon_speed_cm_ns = 28

    rpc_heights = {
        'rpc0': 0, 
        'rpc1': 0.5, 
        'rpc2': 1.0, 
        'rpc3': 61.5, 
        'rpc4': 121.5, 
        'rpc5': 122.0
    }

    for event_number, event_group in tqdm(df.groupby('event_number'), desc="Processing Events"):
        for direction in ['eta', 'phi']:
            direction_group = event_group[event_group['strip_direction'] == direction]
            all_clusters = []

            unique_rpcs = direction_group['rpc_number'].unique()
            for rpc in unique_rpcs:
                rpc_group = direction_group[direction_group['rpc_number'] == rpc]
                for _, row in rpc_group.iterrows():
                    location_scaling = 3.09375 if direction == 'eta' else 2.8125
                    strip_locations = np.array(row['locations'])
                    non_zero_locations = strip_locations[strip_locations != 0]
                    if non_zero_locations.size > 0:
                        strip_location = non_zero_locations[0] 
                    else:
                        continue 

                    location = strip_location * location_scaling
                    event_time = np.mean(row['times']) - rpc_time_offsets[(rpc, direction)][0]
                    cluster_size_scaled = max(row['size'] * location_scaling, location_scaling) / 2
                    all_clusters.append({
                        'rpc': rpc,
                        'location': location,
                        'event_time': event_time,
                        'uncertainty': cluster_size_scaled,
                        'original_location': strip_location
                    })

            combination_metrics = []
            valid_combinations = [] 
            # for n in range(3, 3): 
            for combo in combinations(all_clusters, 3):
                used_rpcs_set = {c['rpc'] for c in combo}
                
                if not used_rpcs_set.issubset({'rpc0', 'rpc1', 'rpc2'}):
                    metric = calculate_metric_for_combo(combo, rpc_heights)
        
                    # Check RSS threshold here and ensure RPC combination uniqueness
                    if metric[4] <= 6000 and len(used_rpcs_set) == len(combo):
                        combination_metrics.append(metric)
                        
            # Move filtering logic outside the loop so it's not reset each time
            for combo_metric in combination_metrics:
                combo = combo_metric[-1]
                if len({c['rpc'] for c in combo}) < len(combo):
                    continue

                time_diffs_are_valid = True
                for i in range(len(combo)):
                    for j in range(i + 1, len(combo)):
                        error_window = rpc_time_offsets[(combo[i]['rpc'], direction)][1] + rpc_time_offsets[(combo[j]['rpc'], direction)][1]
                        
                        # Use direct height differences
                        height_diff = abs(rpc_heights[combo[i]['rpc']] - rpc_heights[combo[j]['rpc']])
                        
                        time_diff = abs(combo[i]['event_time'] - combo[j]['event_time'])
                        expected_time_diff = height_diff / adjusted_muon_speed_cm_ns

                        uncertainty_margin = 5
                        # Use the expected time difference with the error window and uncertainty margin for validation
                        if not (time_diff <= expected_time_diff + error_window + uncertainty_margin):
                            time_diffs_are_valid = False
                            break
                    if not time_diffs_are_valid:
                        break

                if time_diffs_are_valid:
                    valid_combinations.append(combo_metric)

            for valid_combination in valid_combinations:
                paths.append({
                    'Event Number': event_number,
                    'Direction': direction,
                    'Slope': valid_combination[0],
                    'Intercept': valid_combination[1],
                    'Slope_error': valid_combination[2],
                    'Intercept_error': valid_combination[3],
                    'Used Coordinates': [(c['rpc'], c['original_location'], c['event_time']) for c in valid_combination[-1]],
                    'RSS': valid_combination[4],
                })

    path_df = pd.DataFrame(paths)
    return path_df

In [21]:
path_df = analyze_inter_rpc_hit_with_timing_adjusted(fully_massaged_df_with_time)

Processing Events: 100%|██████████| 645/645 [00:02<00:00, 224.96it/s]


In [22]:
show(path_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [23]:
def filter_paths(df):
    excluded_triplet = {'rpc0', 'rpc1', 'rpc2'}

    rows_to_drop = []

    for index, row in df.iterrows():
        used_rpcs = set(c[0] for c in row['Used Coordinates'])

        if used_rpcs.issubset(excluded_triplet):
            rows_to_drop.append(index)  
        elif row['RSS'] > 5:
            rows_to_drop.append(index) 
    
    # Drop the rows from the DataFrame
    filtered_df = df.drop(rows_to_drop).reset_index(drop=True)
    
    return filtered_df

In [24]:
filtered_dataframe = filter_paths(path_df)

show(filtered_dataframe)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [15]:
eta_df = filtered_dataframe[filtered_dataframe['Direction'] == 'eta']
angles_y_axis = 90 - np.degrees(np.arctan(eta_df['Slope']))
angles_y_axis_adjusted = np.where(angles_y_axis > 90, angles_y_axis - 180, angles_y_axis)

plt.figure(figsize=(10, 6))
plt.hist(angles_y_axis_adjusted, bins=20, color='skyblue', edgecolor='black')
plt.xlabel('angles (degrees)')
plt.ylabel('Occurance')
plt.title('Cross_Chamber_Angular_Distribution')
plt.grid(axis='y', alpha=0.75)
plt.show()

NameError: name 'filtered_dataframe' is not defined

In [ ]:
phi_df = filtered_dataframe[filtered_dataframe['Direction'] == 'phi']
angles = np.degrees(np.arctan(phi_df['Slope']))

plt.figure(figsize=(10, 6))
plt.hist(angles, bins=200, color='skyblue', edgecolor='black')
plt.xlabel('angles (degrees)')
plt.ylabel('Occurance')
plt.title('Cross_Chamber_Angular_Distribution_phi')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [19]:
show(filtered_dataframe)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI
